In [1]:
%load_ext autoreload
%autoreload 2

import os
os.environ['KMP_WARNINGS'] = 'off'
import sys
import git

import uproot as ut
import awkward as ak
import numpy as np
import math
import vector
import sympy as sp

import re
from tqdm import tqdm
import timeit
import re

sys.path.append( git.Repo('.', search_parent_directories=True).working_tree_dir )
from utils import *

In [2]:
from functools import partial

In [3]:
module = fc.eightb.feynnet

In [4]:
files = module.feynnet_signal_list[:10]

In [5]:
f_signal = ObjIter(files)

In [6]:
open_tree = partial(Tree, altfile='test_{base}', report=False)

In [7]:
# signal = f_signal.apply(open_tree, report=True)
# del signal

In [8]:
signal = f_signal.pool_apply(open_tree, report=True)

100%|██████████| 10/10 [00:34<00:00,  3.42s/it]


In [9]:
class parallel_reconstruct(ParallelMethod):
    def start(self, tree):
        jet_p4 = build_p4(tree, prefix='jet')

        return dict(
            jet_p4 = jet_p4,
        )
    
    def run(self, jet_p4):
        j1, j2 = jet_p4[:,::2], jet_p4[:,1::2]
        h = j1 + j2
        h1, h2 = h[:,::2], h[:,1::2]
        y = h1 + h2
        y1, y2 = y[:,0], y[:,1]
        x = y1 + y2
        return dict(
            **{f'h_{kin}': getattr(h,kin) for kin in ('pt', 'eta', 'phi', 'm')},
            **{f'y_{kin}': getattr(y,kin) for kin in ('pt', 'eta', 'phi', 'm')},
            **{f'x_{kin}': getattr(x,kin) for kin in ('pt', 'eta', 'phi', 'm')},
        )
    
    def end(self, tree, **output):
        tree.extend(**output)

parallel_reconstruct = parallel_reconstruct()

In [10]:
import multiprocess as mp
with mp.Pool(4) as pool:
    signal.parallel_apply( parallel_reconstruct, pool=pool, report=True)

UnboundLocalError: local variable 'result' referenced before assignment

In [ ]:
signal.x_m

[<Array [488, 497, 449, ... 571, 529, 1.16e+03] type='87927 * float32'>, <Array [513, 500, 531, 500, ... 657, 533, 681] type='89428 * float32'>, <Array [510, 832, 569, ... 1.61e+03, 966] type='91849 * float32'>, <Array [564, 633, 638, ... 1.18e+03, 1.29e+03] type='526600 * float32'>, <Array [581, 529, 603, ... 971, 1.02e+03] type='67945 * float32'>, <Array [1.14e+03, 821, 863, ... 734, 1.35e+03] type='89118 * float32'>, <Array [1.11e+03, 638, 1.14e+03, ... 828, 632] type='564793 * float32'>, <Array [867, 958, 1.24e+03, ... 565, 950] type='580639 * float32'>, <Array [578, 806, 711, ... 651, 815, 1.04e+03] type='108982 * float32'>, <Array [2.24e+03, 889, 920, ... 926, 1.07e+03] type='93134 * float32'>]